In [65]:
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm
import pandas as pd

Генерируем ссылки на страницы с отзывами

In [10]:
links = ['https://www.banki.ru/services/responses/bank/tcs/?is_countable=on']

In [11]:
for i in range(2, 20):
    links.append('https://www.banki.ru/services/responses/bank/tcs/?is_countable=on&page={}&isMobile=0'.format(i))

Генерируем ссылки на страницы с отзывами для тестовой выборки

In [62]:
test_links = []
for i in range(21, 30):
    test_links.append('https://www.banki.ru/services/responses/bank/tcs/?is_countable=on&page={}&isMobile=0'.format(i))

Собираем ссылки на страницы с отзывами

In [34]:
good_links = []
bad_links = []
for l in tqdm(links):
    res = requests.get(l)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    reviews = soup.find_all('article', class_='responses__item')
    for r in reviews:
        rating = r.find('div', class_='flexbox flexbox--inline flexbox--row flexbox--gap_xsmall flexbox--align-items_baseline')
        spans = rating.find_all('span')
        rating = spans[1].text
        tag = r.find('a', class_='header-h3', href=True)
        if int(rating) > 3:
            good_links.append('https://www.banki.ru{}'.format(tag.get('href')))
        elif int(rating) < 3:
            bad_links.append('https://www.banki.ru{}'.format(tag.get('href')))

100%|██████████| 19/19 [00:11<00:00,  1.63it/s]


Собираем ссылки на страницы с отзывами для тестовой выборки

In [64]:
good_test_links = []
bad_test_links = []
for tl in tqdm(test_links):
    res = requests.get(tl)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    reviews = soup.find_all('article', class_='responses__item')
    for r in reviews:
        rating = r.find('div', class_='flexbox flexbox--inline flexbox--row flexbox--gap_xsmall flexbox--align-items_baseline')
        spans = rating.find_all('span')
        rating = spans[1].text
        tag = r.find('a', class_='header-h3', href=True)
        if int(rating) > 3:
            good_test_links.append('https://www.banki.ru{}'.format(tag.get('href')))
        elif int(rating) < 3:
            bad_test_links.append('https://www.banki.ru{}'.format(tag.get('href')))

100%|██████████| 9/9 [00:07<00:00,  1.16it/s]


Создаем корпуса с текстами хороших и плохих отзывов:

In [35]:
with open('bad_texts.txt', 'w', encoding='utf-8') as f:
    f.write('')
with open('good_texts.txt', 'w', encoding='utf-8') as ff:
    ff.write('')

In [36]:
print(len(good_links), len(bad_links))

262 201


In [37]:
for gl in tqdm(good_links):
    res = requests.get(gl)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    rev = soup.find('div', class_='article-text response-page__text markup-inside-small markup-inside-small--bullet')
    with open('good_texts.txt', 'a', encoding='utf-8') as file:
        file.write(rev.text)

for bl in tqdm(bad_links):
    res = requests.get(bl)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    rev = soup.find('div', class_='article-text response-page__text markup-inside-small markup-inside-small--bullet')
    with open('bad_texts.txt', 'a', encoding='utf-8') as file:
        file.write(rev.text)

100%|██████████| 201/201 [01:19<00:00,  2.54it/s]


Создаем датафрейм для тестовой выборки

In [76]:
df_links = []
df_reviews = []
df_polarities = []

In [75]:
good_test_reviews = []
for gtl in tqdm(good_test_links):
    res = requests.get(gtl)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    rev = soup.find('div', class_='article-text response-page__text markup-inside-small markup-inside-small--bullet')
    good_test_reviews.append(rev.text)

bad_test_reviews = []        
for btl in tqdm(bad_test_links):
    res = requests.get(btl)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    rev = soup.find('div', class_='article-text response-page__text markup-inside-small markup-inside-small--bullet')
    bad_test_reviews.append(rev.text)

100%|██████████| 47/47 [00:32<00:00,  1.45it/s]


In [77]:
for gtl in range(len(good_test_links)):
    df_links.append(good_test_links[gtl])
    df_polarities.append('good')
    df_reviews.append(good_test_reviews[gtl])

for btl in range(len(bad_test_links)):
    df_links.append(bad_test_links[btl])
    df_polarities.append('bad')
    df_reviews.append(bad_test_reviews[btl])

In [78]:
df = pd.DataFrame({
    'review_link': df_links,
    'review_text': df_reviews,
    'polarity': df_polarities
})

In [79]:
df.head(5)

,review_link,review_text,polarity
0,https://www.banki.ru/services/responses/bank/r...,\n\t\tВ ноябре 3 числа решил оформить себе деб...,good
1,https://www.banki.ru/services/responses/bank/r...,\n\t\tЯ уже довольно давно пользуюсь услугами ...,good
2,https://www.banki.ru/services/responses/bank/r...,\n\t\tЭто произошло 22 июня 2021. Я совершил п...,good
3,https://www.banki.ru/services/responses/bank/r...,\n\t\tПишу отзыв о наилучшем банке России по м...,good
4,https://www.banki.ru/services/responses/bank/r...,\n\t\tЯ начал пользоваться Тинькофф банком ещё...,good


In [84]:
df = df.sample(frac=1).reset_index(drop=True)

In [85]:
df.head(5)

,review_link,review_text,polarity
0,https://www.banki.ru/services/responses/bank/r...,"\n\t\tПользуясь многими банками в стране, хоте...",good
1,https://www.banki.ru/services/responses/bank/r...,"\n\t\tПользуюсь банком уже 3 года, за все врем...",good
2,https://www.banki.ru/services/responses/bank/r...,\n\t\tНачну с небольшой предыстории. Клиентом ...,good
3,https://www.banki.ru/services/responses/bank/r...,"\n\t\t12.08 оплатил полис ОСАГО, авторизовалис...",bad
4,https://www.banki.ru/services/responses/bank/r...,\n\t\t12 августа пытался воспользоваться банко...,bad


In [38]:
from nltk import word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [39]:
with open('good_texts.txt', 'r', encoding='utf-8') as file:
    good_corpus = file.read()
    
with open('bad_texts.txt', 'r', encoding='utf-8') as file:
    bad_corpus = file.read()   

Создаем функцию, отвечающую за препроцессинг (токенизирует, приводит к нижнему регистру и приводит слова в начальную форму)

In [40]:
def preprocess(text):
    text = text.lower()
    words = word_tokenize(text)
    out = []
    for w in words:
        p = morph.parse(w)[0]
        out.append(p.normal_form)
    return out

In [41]:
preprocessed_good = preprocess(good_corpus)
preprocessed_bad = preprocess(bad_corpus)

In [43]:
preprocessed_good[:10]

['спасибо',
 'банк',
 'за',
 'лояльность',
 'и',
 'оперативный',
 'решение',
 'вопрос',
 '!',
 'решить']

Убираем из корпусов слова, встретившиеся меньше четырех раз и создаем множества с уникальными для корпусов словами

In [44]:
from collections import Counter

In [45]:
good_c = Counter(preprocessed_good)
bad_c = Counter(preprocessed_bad)

In [46]:
good_c.most_common(10)

[(',', 3251),
 ('.', 1930),
 ('и', 1349),
 ('в', 1328),
 ('я', 1144),
 ('банк', 828),
 ('на', 693),
 ('не', 663),
 ('что', 537),
 ('с', 531)]

In [47]:
good_corp = []
for k, v in good_c.items():
    if v > 3:
        good_corp.append(k)
        
bad_corp = []
for k, v in bad_c.items():
    if v > 3:
        bad_corp.append(k)

In [48]:
len(good_corp), len(bad_corp)

(1172, 1293)

In [49]:
good_corp = set(good_corp)
bad_corp = set(bad_corp)

In [50]:
unique_b = bad_corp - good_corp
unique_g = good_corp - bad_corp

In [52]:
len(unique_b), len(unique_g)

(480, 359)

Создаем функцию, определяющую полярность отзыва (положительный / отрицательный)

In [53]:
def guess(review):
    preprocessed_r = preprocess(review)
    review_words = set(preprocessed_r)
    bad_intersection = review_words & unique_b
    good_intersection = review_words & unique_g
    if len(bad_intersection) > len(good_intersection):
        return 'bad'
    elif len(good_intersection) > len(bad_intersection):
        return 'good'
    else:
        return 'unsure'

Проверяем работу функции

In [90]:
predictions = []
accuracy = []
for i in range(len(df['review_text'])):
    prediction = guess(df['review_text'][i])
    predictions.append(prediction)
    if df['polarity'][i] == prediction:
        accuracy.append(1)
    else:
        accuracy.append(0)

In [91]:
df['predictions'] = predictions
df['accuracy'] = accuracy

In [92]:
df.head(5)

,review_link,review_text,polarity,predictions,accuracy
0,https://www.banki.ru/services/responses/bank/r...,"\n\t\tПользуясь многими банками в стране, хоте...",good,good,1
1,https://www.banki.ru/services/responses/bank/r...,"\n\t\tПользуюсь банком уже 3 года, за все врем...",good,good,1
2,https://www.banki.ru/services/responses/bank/r...,\n\t\tНачну с небольшой предыстории. Клиентом ...,good,good,1
3,https://www.banki.ru/services/responses/bank/r...,"\n\t\t12.08 оплатил полис ОСАГО, авторизовалис...",bad,bad,1
4,https://www.banki.ru/services/responses/bank/r...,\n\t\t12 августа пытался воспользоваться банко...,bad,bad,1


In [95]:
accuracy_df = df['accuracy'].value_counts()

In [96]:
accuracy = accuracy_df[1] / (accuracy_df[0] + accuracy_df[1])

In [97]:
accuracy

0.8482142857142857

Итак, accuracy модели получилось приблизительно 0,85.

Для улучшения программы можно:
1. Учитывать частотность лексем, а не только факт пересечения с тем или иным множеством
2. Учитывать длину отзыва (хорошие отзывы обычно короче, чем плохие)
3. Брать только отзывы с одной звездой для корпуса негативных отзывов и только отзывы с пятью звездами для корпуса позитивных (так они будут более эмоционально заряженными). 